In [1]:
import requests
import pandas as pd
import datetime

In [5]:
import requests

api_key = "fjbIvblscE9aaoZhczJxUaZQuJAeGtsu" 
from urllib.parse import quote

def get_coordinates(address, api_key):
    encoded_address = quote(address)
    url = f"https://api.tomtom.com/search/2/geocode/{encoded_address}.json?key={api_key}"
    
    response = requests.get(url)
    data = response.json()

    if data["results"]:
        position = data["results"][0]["position"]
        lat = position["lat"]
        lon = position["lon"]
        return lat, lon
    else:
        return None, None

address = "15 Bhaktivedanta Swami Road, Vile Parle West, Mumbai, Maharashtra 400056"


lat, lon = get_coordinates(address, api_key)
print(f"Latitude: {lat}, Longitude: {lon}")


Latitude: 19.0989941, Longitude: 72.8395499


In [ ]:
import requests

def get_traffic_flow(lat, lon, api_key):
    url = "https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json"
    params = {
        "point": f"{lat},{lon}",
        "key": api_key
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        print(data)
        # print("🚗 Traffic Flow Info:")
        # current_speed = data['flowSegmentData']['currentSpeed']
        # free_flow_speed = data['flowSegmentData']['freeFlowSpeed']
        # confidence = data['flowSegmentData']['confidence']
        # road_class = data['flowSegmentData']['frc']
        
        # print(f"- Current Speed: {current_speed} km/h")
        # print(f"- Free Flow Speed: {free_flow_speed} km/h")
        # print(f"- Confidence: {confidence}")
        # print(f"- Road Class: {road_class}")
    else:
        print("❌ Error fetching traffic data:", response.status_code)

# Coordinates and API key
lat = 19.0989941
lon = 72.8395499
api_key = "fjbIvblscE9aaoZhczJxUaZQuJAeGtsu"

# Call the function
get_traffic_flow(lat, lon, api_key)
